In [1]:
# Import des bibliothèques
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Import des fichiers flux Darva et Editel
df1 = pd.read_excel('darva2.xlsx')
df2 = pd.read_excel('editel2.xlsx')

#----------Modifier les colonnes val_suffix_sinistre et num_sini--------

df2a = df2.rename(columns={'num_sini' : 'NUM_SINI', 'val_suffix_sinistre' : 'VAL_SUFFIX_SINISTRE'})

#---------------Gestion des 0 manquants avec le format xlsx-------------
# Darva:
# Modif Colonne noMission:
def ajouter_zeros1(valeur, longueur_totale=10):
    if pd.notna(valeur):
        return str(valeur).zfill(longueur_totale) # Vérifie si la valeur n'est pas NaN
    return valeur
# Appliquer la fonction à la colonne 
df1['noMission'] = df1['noMission'].apply(ajouter_zeros1)

# Modif Colonne noSinistre:
def ajouter_zeros2(valeur, longueur_totale=16):
    if pd.notna(valeur):
        return str(valeur).zfill(longueur_totale) # Vérifie si la valeur n'est pas NaN
    return valeur
# Appliquer la fonction à la colonne 
df1['noSinistre'] = df1['noSinistre'].apply(ajouter_zeros2)

# Editel:
# Modif Colonne val_suffix_sinistre:
df2a['VAL_SUFFIX_SINISTRE'] = df2a['VAL_SUFFIX_SINISTRE'].apply(ajouter_zeros1)

# Modif Colonne num_sini:
df2a['NUM_SINI'] = df2a['NUM_SINI'].apply(ajouter_zeros2)


#-----------------------------------------------------------------------

#Création des masques pour travailler les données
df1_clean = df1.copy()
editel_clean = df2a.copy()

#Suppression des champs inutiles
# Darva:
df1_clean.drop(['idInterneLot','numLot','dateHeureDepotLot','indicTraiteLot','codeDomaine','natureCodeAssureur','codeComplementaireAssureur','idExterneMessage','idInterneMessage','refMissionPrestataire',
               'noMessage','noFacture','refReglement','idPJ', 'Unnamed: 26', 'Unnamed: 27'], axis=1, inplace=True)

#Editel:
editel_clean.drop(['NUM_CHRONO_TEXT_ENV'], axis=1, inplace=True)

# Conversion des colonnes date
# Darva:
df1_clean['dateHeureDepotMessage'] = pd.to_datetime(df1_clean['dateHeureDepotMessage'])
df1_clean['dateHeureRecuperationMessage'] = pd.to_datetime(df1_clean['dateHeureRecuperationMessage'])

# Conversion des colonnes 
# Darva:
df1_clean['codeAssureur'] = df1_clean['codeAssureur'].astype(str)
df1_clean['noSinistre'] = df1_clean['noSinistre'].astype(str)
df1_clean['noMission'] = df1_clean['noMission'].astype(str)
df1_clean['erreur'] = df1_clean['erreur'].astype('category')

# Editel
editel_clean['NUM_DOS'] = editel_clean['NUM_DOS'].astype(str)
editel_clean['NUM_SINI'] = editel_clean['NUM_SINI'].astype(str)
editel_clean['VAL_SUFFIX_SINISTRE'] = editel_clean['VAL_SUFFIX_SINISTRE'].astype(str)
editel_clean['NUM_MIS'] = editel_clean['NUM_MIS'].astype(str)
editel_clean['COD_DOC'] = editel_clean['COD_DOC'].astype(str)
editel_clean['COD_MES'] = editel_clean['COD_MES'].astype(str)
editel_clean['COD_SOC'] = editel_clean['COD_SOC'].astype(str)
editel_clean['Intervenant Télématique'] = editel_clean['Intervenant Télématique'].astype(str)

# Concaténation des colonnes N°sin/N°mission en identifiantDarva pour créer la clé entre les deux df
# Darva:
df1_clean['identifiantDarva'] = df1_clean['noSinistre'] + '-' + df1_clean['noMission']

# Editel:
editel_clean['identifiantDarva'] = editel_clean['NUM_SINI'] + '-' + editel_clean['VAL_SUFFIX_SINISTRE']

# Suppression des doublons pour chaque df
#df1_clean = df1_clean.drop_duplicates(subset ='identifiantDarva')
#editel_clean = editel_clean.drop_duplicates(subset ='identifiantDarva')

# Suppression de caractères dans la colonne erreur
# Darva:
df1_clean['erreur'] = df1_clean['erreur'].str.slice(start=0, stop=3)

# Mapping des référentiels
#(Création du dictionnaire d'erreur (Référentiel d'erreurs) pour mapper les clefs avec les codes erreurs de la colonne df_clean.erreur)
# Darva:
erreur_dictionary ={'C01' : 'Bloquant OM absent', 'C02' : 'Bloquant OM Hors contexte', 'C03' : 'Bloquant OM en double',
                    'C04' : 'Bloquant RE sans OM', 'C07' : 'Bloquant NH sans OM', 'C08' : 'Bloquant NH hors contexte', 'C0A' : 'Bloquant DE sans OM',
                    'C0B' : 'Bloquant AS absent', 'C0I' : ' OM à générer déjà existant', 'C15' : 'Bloquant Action PJ impossible à cause du contexte',
                    'C16' : 'Warning Document cible non trouvé', 'C17' : 'Bloquant Document cible inactif',
                    'C18' : 'Bloquant Demande ajout d\'une pièce jointe qui est déjà liée', 'C33' : 'Bloquant CA sans ET', 'C52' : 'Bloquant Dossier absent',
                    'C53' : 'Warning Dossier archivé', 'C70' : 'Bloquant Dossier absent avec scénario', 'C73' : 'Bloquant OM avec scénario différent du scénario du dossier',
                    'C77' : 'Bloquant Destinataire de la copie par scénario non trouvé', 'C78' : 'Bloquant Scénario document valide mais dossier en erreur scénario',
                    'C79' : 'Bloquant Document avec scénario mais dossier sans scénario', 'C7A' : 'Bloquant Scénario du document n\'est pas précisé alors que dossier avec scénario',
                    'C7D' : 'Warning Document à désarchiver préalablement', 'C7E' : 'Bloquant Type d\'échange interdit par le scénario', 'C96' : 'Bloquant SE sans DE',
                    'C9K' : 'Bloquant Le partenaire  destinataire du document n\'intervient pas sur le dossier', 'C9L' : 'Bloquant IFR sanas AS',
                    'D05' : 'Bloquant Emetteur et/ou destinataire invalide pour ce type de document', 'F02' : 'Warning stop circuit', 'F04' : 'Warning Nature de mission interdite',
                    'F05' : 'Warning modalité de détermination des dommages interdite', 'F06' : 'Warning Autorisation de la modalité interdite', 'F07' : 'Warning Nature de l\'événement interdite',
                    'F08' : 'Warning Nature du véhicule adverse ou de l\'obstacle interdite', 'F09' : 'Warning Règlement direct consenti interdit',
                    'F10' : 'Warning Convention appliquée interdite', 'F11' : 'Warning Pourcentage de responsabilité invalide', 'F12' : 'Warning Genre du véhicule',
                    'F14' : 'Warning Pluralité des chocs', 'F15' : 'Warnin!g Code du réparateur chez l\'assureur absent', 'F16' : 'Warning Réparateur non autorisé',
                    'F20' : 'Warning Montant total HT invalide (IDA)', 'F21' : 'Warning Nombre estimation maximal atteint', 'F22' : 'Warning Combinaison invalide flitre combiné',
                    'G07' : 'Bloquant Identifiant MM en double', 'GA3' : 'Bloquant Code GTA erroné', 'GB4' : 'Bloquant Code abonné noueau réparateur non renseigné',
                    'GB5' : 'Bloquant Code abonné nouveau réparateur non différent', 'GB6' : 'Bloquant Code abonné nouveau réparateur inconnu',
                    'GB7' : 'Bloquant Code abonné nouveau réparateur de type erroné', 'GCA' : 'Bloquant ', 'GDT' : 'Warning Assureur non autorisé à l\'utilisation des SD',
                    'GR6' : 'Bloquant (pas de libellé erreur)', 'I41' : 'Warning Annulation de mission après SD76', 'I44' : 'Warning Le code abonné de la donnée DE0820 du segment 160 n\'est pas valide',
                    'L04' : 'Bloquant Lot en double', 'P30' : 'Warning Absence estimation des travaux dans les délais après réception de l\'avis de sinistre',
                    'P40' : 'Warning Absence estimation des travaux dans les délais après réception du dernier compte-rendu d\'analyse',
                    'P60' : 'Warning Absence faction de réparation ou de l\'estimation des travaux dans les délais après rejet de la dernière faction de réparation',
                   }
df1_clean['type erreur'] = df1_clean['erreur'].map(erreur_dictionary)

# Editel:
# 1 Concaténer les colonnes COD_DOC et COD_MES pour mapper les types de messages
editel_clean['ReferenceMessageDarva'] = editel_clean['COD_DOC'] + '.' + editel_clean['COD_MES']

# 2 Création du dictionnaire des messages Darva
message_darva ={'41' : 'SD', '15' : 'RE', '20' : 'NH', '14' : 'CH', '61' : 'BE'}

type_mess = {'41.0': 'SUIVI DE DOSSIER ','41.02':'REMISE A DISPOSITION ', '41.03':'MODIFICATION DE L\'ORDRE DE MISSION', '41.04':'CONTESTABILITE DE L\'EXPERTISE',
                  '41.05' : 'MODIFICATION DE PRISE EN CHARGE', '41.09' : 'REPONSE OFFRE DE CESSION', '41.10' : 'VEHICULE HORS SECTEUR',
                   '41.10' : 'VEHICULE HORS SECTEUR ', '41.11' : 'EXPERT INDISPONIBLE ', '41.12' : 'CLOTURE DOSSIERS SPECIAUX ',
                   '41.13' : 'CARENCE DU LESE ', '41.15' : 'RETARD PREVISIBLE ', '41.16' : 'ACCUSE DE RECEPTION ',
                   '41.18' : 'INFORMATION SUR LA PROCEDURE V.E.', '41.19' : 'CERTIFICAT DE CONFORMITE ', '41.20' : 'RELANCE DE L\'EXPERT ',
                  '41.21' : 'DESIGNATION OU CHANGEMENT D\'EXPERT ', '41.22' : 'INFORMATION SUR LE MONTANT DE LA REPARATION ',
             '41.23' : 'VEHICULE RETROUVE ', '41.24' : 'FICHE CONSTATATION VOL ', '41.25' : 'ANNULATION DE L\'ORDRE DE MISSION',
             '41.26' : 'VEHICULE INDEMNISE', '41.29' : 'INFORMATIONS SUR LES DOMMAGES ', '41.31' : 'INFORMATION BAQUET',
             '41.32' : 'DESIGNATION OU CHANGEMENT DE REPARATEUR', '41.33' : 'MODIFICATION DE L\'AVIS DE SINISTRE ',
             '41.35' : 'ANNULATION DE L\'AVIS DE SINISTRE', '41.36' : 'VALIDATION DE LA FACTURE', '41.37' : 'REJET DE LA FACTURE',
            '41.38' : 'DEMANDE DE L\'ORDRE DE MISSION', '41.39' : 'DEMANDE D\'OUVERTURE DE DOSSIER',
             '41.3A' : 'MODIFICATION DE L\'ORDRE DE MISSION pour Argos', '41.40' : 'PRISE DE RENDEZ VOUS',
             '41.41': 'QUALIFICATION DOSSIER EAD ', '41.42' : 'QUESTIONNAIRE INCENDIE ', '41.50' : 'DESIGNATION OU CHANGEMENT DE REPARATEUR ET/OU D\'EXPERT',
             '41.54' : 'AVIS D\'EXPERTISE', '41.60' : 'GESTION RECOURS', '41.74' : 'ELIGIBILITE CHIFFRAGE REPARATEUR',
             '41.75' : 'REJET CHIFFRAGE REPARATEUR', '41.76' : 'CONFIRMATION CHIFFRAGE REPARATEUR', '41.77' : 'CHANGEMENT DU NUMERO DE MISSION ',
             '41.85' : 'TMA', '41.99' : 'COMMENTAIRES' , '15.0' : 'RE', '20.0' : 'NOTE D\'HONORAIRES', '14.0' : 'CHIFFRAGE'}

editel_clean['message'] = editel_clean['COD_DOC'].map(message_darva)

editel_clean['type message'] = editel_clean['ReferenceMessageDarva'].map(type_mess)

# 3 Suppression des colonnes ('COD_DOC','COD_MES') du dataframe editel_clean car inutile après la création de la colonne ReferenceMessageDarva
#editel_clean = editel_clean.drop(['COD_DOC','COD_MES'], axis=1)


#------------------------------------RE--------------------------------------------

# Application du filtre BCA (flux darva sortants: BCA vers Darva)
df_bca = df1_clean.query('emetteurMessage =="E92000011600"')

# Afficher les différents types de message BCA (utile pour trouver les BE)
#df_bca.typeMessage.value_counts()

# Application du filtre rapports d'expertise
#Darva:
rapport_expertise = df_bca.query('typeMessage =="RAPPORT D\'EXPERTISE"')

#Editel:
RE_editel = editel_clean.query('COD_DOC =="15"')

#Nb total de rapports d'expertise
#a = rapport_expertise.shape[0]

#Nb total de numéro uniques de sinistres
#b = rapport_expertise['identifiantDarva'].nunique()

#Nombre de doublons
#c = sum(rapport_expertise['identifiantDarva'].duplicated())

#print("Pour {} rapports d'expertises, nous avons {} rapports uniques et {} rapports en doublon.".format(a,b,c))

# Suppression des doublons dans la liste des rapports
rapport_unique = rapport_expertise.drop_duplicates(subset ='identifiantDarva')

# Application du filtre rapports en erreur sur les rapports uniques
#rapport_echec = rapport_unique.query('indicTraiteMessage =="KO"')

# Comptabilisation des erreurs
#succes = rapport_unique.shape[0]
#echec = rapport_echec.shape[0]
#print("Pour {} rapports d'expertises envoyés, nous avons {} rapports retournés en erreur.".format(succes,echec))

# RE issu du flux Darva présents dans Editel
RE_darva_editel_present = pd.merge(rapport_unique, RE_editel, how='inner', on='identifiantDarva')

# RE issu du flux Darva absents dans Editel (temps réel) (à prendre)
RE_darva_editel_absent = pd.merge(rapport_unique, RE_editel, how='outer', on='identifiantDarva', indicator=True).query('_merge=="left_only"').drop(columns='_merge')

# Données Editel n'ayant pas de correspondance avec le flux Darva (Editel manquant) (à prendre)
RE_editel_sans_RE_Darva =pd.merge(rapport_unique, RE_editel, how='outer', on='identifiantDarva', indicator=True).query('_merge=="right_only"').drop(columns='_merge')


#-----------------------------------------------NH-----------------------------------------------------------

# Application du filtre note d'honoraires
#Darva:
note_honoraire = df_bca.query('typeMessage =="NOTE D\'HONORAIRES"')

#Editel:
NH_editel = editel_clean.query('COD_DOC =="20"')

#Nb total de notes d'honoraires
#note = note_honoraire.shape[0]

#Nb total de NH uniques 
#nh_id = note_honoraire['identifiantDarva'].nunique()

#Identification des doublons
#nh_doublon = sum(note_honoraire['identifiantDarva'].duplicated())

#print("Pour {} notes d'honoraires, nous avons {} NH uniques et {} NH en doublon.".format(note,nh_id,nh_doublon))

# Suppression des doublons dans la liste des NH
nh_unique = note_honoraire.drop_duplicates(subset ='identifiantDarva')

# Application du filtre NH en erreur sur les NH uniques
#nh_echec = nh_unique.query('indicTraiteMessage =="KO"')

# Comptabilisation des erreurs
#nh_succes = nh_unique.shape[0]
#nh_ko = nh_echec.shape[0]
#print("Pour {} notes d'honoraires uniques envoyés, nous avons {} NH retournés en erreur.".format(nh_succes,nh_ko))

# NH issu du flux Darva présents dans Editel
NH_darva_editel_present = pd.merge(nh_unique, NH_editel, how='inner', on='identifiantDarva')

# NH issu du flux Darva absents dans Editel (temps réel) (à prendre)
NH_darva_editel_absent = pd.merge(nh_unique, NH_editel, how='outer', on='identifiantDarva', indicator=True).query('_merge=="left_only"').drop(columns='_merge')

# NH Données Editel n'ayant pas de correspondance avec le flux Darva (Editel manquant) (à prendre)
NH_editel_sans_NH_Darva =pd.merge(nh_unique, NH_editel, how='outer', on='identifiantDarva', indicator=True).query('_merge=="right_only"').drop(columns='_merge')


#------------------------------------------------BE------------------------------------------------------------

# Application du filtre INFORMATION SUR LA DEMANDE D'ENLEVEMENT
#Darva:
bon_enlevement = df_bca.query('typeMessage =="INFORMATION SUR LA DEMANDE D\'ENLEVEMENT"')

#Editel:
BE_editel = editel_clean.query('COD_DOC =="61"')

# Suppression des doublons dans la liste des BE
be_unique = bon_enlevement.drop_duplicates(subset ='identifiantDarva')

# BE issu du flux Darva présents dans Editel
BE_darva_editel_present = pd.merge(be_unique, BE_editel, how='inner', on='identifiantDarva')

# BE issu du flux Darva absents dans Editel (temps réel) (à prendre)
BE_darva_editel_absent = pd.merge(be_unique, BE_editel, how='outer', on='identifiantDarva', indicator=True).query('_merge=="left_only"').drop(columns='_merge')

# Données Editel n'ayant pas de correspondance avec le flux Darva (Editel manquant) (à prendre)
BE_editel_sans_BE_Darva =pd.merge(be_unique, BE_editel, how='outer', on='identifiantDarva', indicator=True).query('_merge=="right_only"').drop(columns='_merge')


#--------------------------------------------------Visualisation des messages --------------------------------------------------------------
ar = np.array([['Darva/Editel', RE_darva_editel_present.shape[0], NH_darva_editel_present.shape[0], BE_darva_editel_present.shape[0]],
               ['Temps réel', RE_darva_editel_absent.shape[0], NH_darva_editel_absent.shape[0], BE_darva_editel_absent.shape[0]],
               ['A reprendre', RE_editel_sans_RE_Darva.shape[0], NH_editel_sans_NH_Darva.shape[0], BE_editel_sans_BE_Darva.shape[0]]])
synthese = pd.DataFrame(ar, index = ['Darva/Editel', 'Temps réel', 'A reprendre'], columns = ['','RE', 'NH', 'BE'])

print(synthese)

#------------------------------------------------EXPORT RE--------------------------------------------------------

# Dataframe rapports en doublons
#rapport_doublon = rapport_expertise[rapport_expertise['identifiantDarva'].duplicated()]
#rapport_doublon.to_excel('rapport_doublon.xlsx', sheet_name='RE_doublon', index=False)

# Dataframe rapports uniques
#rapport_unique.to_excel('rapport_unique.xlsx', sheet_name='RE_uniques', index=False)

# Dataframe rapports en erreur
#rapport_echec.to_excel('rapport_echec.xlsx', sheet_name='RE_echec', index=False)

# Rapports d'expertise issu du flux Darva présents dans Editel (OK/KO)
RE_darva_editel_present.to_excel('RE_darva_editel_present.xlsx', sheet_name='RE_present', index=False)

# RE issu du flux Darva absents dans Editel (temps réel) 
RE_darva_editel_absent.to_excel('RE_temps_reel.xlsx', sheet_name='RE_absent', index=False)

# Données Editel n'ayant pas de correspondance avec le flux Darva (Données à reprendre)
RE_editel_sans_RE_Darva.to_excel('RE_Darva_a_reprendre.xlsx', sheet_name='RE_absent', index=False)


#------------------------------------------------EXPORT NH--------------------------------------------------------
# Dataframe NH en doublons
#nh_doublon = note_honoraire[note_honoraire['identifiantDarva'].duplicated()]
#nh_doublon.to_excel('nh_doublon.xlsx', sheet_name='NH_doublons', index=False)

# Dataframe NH uniques
#nh_unique.to_excel('nh_unique.xlsx', sheet_name='NH_uniques', index=False )

# Dataframe NH en erreur
#nh_echec.to_excel('nh_echec.xlsx', sheet_name='NH_echec', index=False)

# NH issu du flux Darva présents dans Editel (OK/KO)
NH_darva_editel_present.to_excel('NH_darva_editel_present.xlsx', sheet_name='NH_present', index=False)

# NH issu du flux Darva absents dans Editel (temps réel) 
NH_darva_editel_absent.to_excel('NH_temps_reel.xlsx', sheet_name='NH_absent', index=False)

# NH Données Editel n'ayant pas de correspondance avec le flux Darva (Données à reprendre)
NH_editel_sans_NH_Darva.to_excel('NH_Darva_a_reprendre.xlsx', sheet_name='NH_absent', index=False)

#------------------------------------------------EXPORT BE--------------------------------------------------------

# BE issu du flux Darva présents dans Editel (OK/KO)
BE_darva_editel_present.to_excel('BE_darva_editel_present.xlsx', sheet_name='BE_present', index=False)

# BE issu du flux Darva absents dans Editel (temps réel) 
BE_darva_editel_absent.to_excel('BE_temps_reel.xlsx', sheet_name='BE_absent', index=False)

# BE Données Editel n'ayant pas de correspondance avec le flux Darva (Données à reprendre)
BE_editel_sans_BE_Darva.to_excel('BE_Darva_a_reprendre.xlsx', sheet_name='BE_absent', index=False)

#------------------------------------------Export Visualisation-----------------------------------------------------
synthese.to_excel('synthese.xlsx', sheet_name='Messages', index=False)

                              RE    NH   BE
Darva/Editel  Darva/Editel   767   565   16
Temps réel      Temps réel  1864  1092  244
A reprendre    A reprendre    31    27    2
